# Performing Database-style Operations on Dataframes

## About the data
In this notebook, we will using daily weather data that was taken from the [National Centers for Environmental Information (NCEI) API](https://www.ncdc.noaa.gov/cdo-web/webservices/v2). The [`0-weather_data_collection.ipynb`](./0-weather_data_collection.ipynb) notebook contains the process that was followed to collect the data. Consult the dataset's [documentation](https://www1.ncdc.noaa.gov/pub/data/cdo/documentation/GHCND_documentation.pdf) for information on the fields.

*Note: The NCEI is part of the National Oceanic and Atmospheric Administration (NOAA) and, as you can see from the URL for the API, this resource was created when the NCEI was called the NCDC. Should the URL for this resource change in the future, you can search for "NCEI weather API" to find the updated one.*


## Setup

In [ ]:
import pandas as pd

weather = pd.read_csv('data/nyc_weather_2018.csv')
weather.head()

## Querying DataFrames
The `query()` method is an easier way of filtering based on some criteria. For example, we can use it to find all entries where snow was recorded:

In [ ]:
snow_data = weather.query('datatype == "SNOW" and value > 0')
snow_data.head()

This is equivalent to querying the `weather.db` SQLite database for 

```sql
SELECT * 
FROM weather 
WHERE datatype == "SNOW" AND value > 0
```

In [ ]:
import sqlite3

with sqlite3.connect('data/weather.db') as connection:
    snow_data_from_db = pd.read_sql(
        'SELECT * FROM weather WHERE datatype == "SNOW" AND value > 0 and station LIKE "%US1NY%"', 
        connection
    )

snow_data.reset_index().drop(columns='index').equals(snow_data_from_db)

Note this is also equivalent to creating Boolean masks:

In [ ]:
weather[
    (weather.datatype == 'SNOW') 
    & (weather.value > 0)
    & weather.station.str.contains('US1NY')
].equals(snow_data)

## Merging DataFrames
We have data for many different stations each day; however, we don't know what the stations are&mdash;just their IDs. We can join the data in the `weather_stations.csv` file which contains information from the `stations` endpoint of the NCEI API. Consult the [`0-weather_data_collection.ipynb`](./0-weather_data_collection.ipynb) notebook to see how this was collected. It looks like this:

In [ ]:
station_info = pd.read_csv('data/weather_stations.csv')
station_info.head()

As a reminder, the weather data looks like this:

In [ ]:
weather.head()

We can join our data by matching up the `station_info.id` column with the `weather.station` column. Before doing that though, let's see how many unique values we have:

In [ ]:
station_info.id.describe()

While `station_info` has one row per station, the `weather` dataframe has many entries per station. Notice it also has fewer uniques:

In [ ]:
weather.station.describe()

When working with joins, it is important to keep an eye on the row count. Some join types will lead to data loss. Remember that we can get this with `shape`:

In [ ]:
station_info.shape[0], weather.shape[0]

Since we will be doing this often, it makes more sense to write a function:

In [ ]:
def get_row_count(*dfs):
    return [df.shape[0] for df in dfs]
get_row_count(station_info, weather)

By default, `merge()` performs an inner join. We simply specify the columns to use for the join. The left dataframe is the one we call `merge()` on, and the right one is passed in as an argument:

In [ ]:
inner_join = weather.merge(station_info, left_on='station', right_on='id')
inner_join.sample(5, random_state=0)

We can remove the duplication of information in the `station` and `id` columns by renaming one of them before the merge and then simply using `on`:

In [ ]:
weather.merge(station_info.rename(dict(id='station'), axis=1), on='station').sample(5, random_state=0)

We are losing stations that don't have weather observations associated with them, if we don't want to lose these rows, we perform a right or left join instead of the inner join:

In [ ]:
left_join = station_info.merge(weather, left_on='id', right_on='station', how='left')
right_join = weather.merge(station_info, left_on='station', right_on='id', how='right')

right_join[right_join.datatype.isna()].head()

The left and right join as we performed above are equivalent because the side for which we kept the rows without matches was the same in both cases:

In [ ]:
left_join.sort_index(axis=1).sort_values(['date', 'station'], ignore_index=True).equals(
    right_join.sort_index(axis=1).sort_values(['date', 'station'], ignore_index=True)
)

Note we have additional rows in the left and right joins because we kept all the stations that didn't have weather observations:

In [ ]:
get_row_count(inner_join, left_join, right_join)

If we query the station information for stations that have `US1NY` in their ID and perform an outer join, we can see where the mismatches occur:

In [ ]:
outer_join = weather.merge(
    station_info[station_info.id.str.contains('US1NY')], 
    left_on='station', right_on='id', how='outer', indicator=True
)

pd.concat([
    outer_join.query(f'_merge == "{kind}"').sample(2, random_state=0) 
    for kind in outer_join._merge.unique()
]).sort_index()

These joins are equivalent to their SQL counterparts. Below is the inner join. Note that to use `equals()` you will have to do some manipulation of the dataframes to line them up:

In [ ]:
import sqlite3

with sqlite3.connect('data/weather.db') as connection:
    inner_join_from_db = pd.read_sql(
        'SELECT * FROM weather JOIN stations ON weather.station == stations.id', 
        connection
    )

inner_join_from_db.shape == inner_join.shape

Revisiting the dirty data from lab 3's [`5-handling_data_issues.ipynb`](../ch_03/5-handling_data_issues.ipynb) notebook.

Data meanings:
- `PRCP`: precipitation in millimeters
- `SNOW`: snowfall in millimeters
- `SNWD`: snow depth in millimeters
- `TMAX`: maximum daily temperature in Celsius
- `TMIN`: minimum daily temperature in Celsius
- `TOBS`: temperature at time of observation in Celsius
- `WESF`: water equivalent of snow in millimeters


Read in the data, dropping duplicates and the uninformative `SNWD` column:

In [ ]:
dirty_data = pd.read_csv(
    'data/dirty_data.csv', index_col='date'
).drop_duplicates().drop(columns='SNWD')
dirty_data.head()

We need to create two dataframes for the join. We will drop some unecessary columns as well for easier viewing:

In [ ]:
valid_station = dirty_data.query('station != "?"').drop(columns=['WESF', 'station'])
station_with_wesf = dirty_data.query('station == "?"').drop(columns=['station', 'TOBS', 'TMIN', 'TMAX'])

Our column for the join is the index in both dataframes, so we must specify `left_index` and `right_index`:

In [ ]:
valid_station.merge(
    station_with_wesf, how='left', left_index=True, right_index=True
).query('WESF > 0').head()

The columns that existed in both dataframes, but didn't form part of the join got suffixes added to their names: `_x` for columns from the left dataframe and `_y` for columns from the right dataframe. We can customize this with the `suffixes` argument:

In [ ]:
valid_station.merge(
    station_with_wesf, how='left', left_index=True, right_index=True, suffixes=('', '_?')
).query('WESF > 0').head()

Since we are joining on the index, an easier way is to use the `join()` method instead of `merge()`. Note that the suffix parameter is now `lsuffix` for the left dataframe's suffix and `rsuffix` for the right one's:

In [ ]:
valid_station.join(station_with_wesf, how='left', rsuffix='_?').query('WESF > 0').head()

Joins can be very resource-intensive, so it's a good idea to figure out what type of join you need using set operations before trying the join itself. The `pandas` set operations are performed on the index, so whichever columns we will be joining on will need to be the index. Let's go back to the `weather` and `station_info` dataframes and set the station ID columns as the index:

In [ ]:
weather.set_index('station', inplace=True)
station_info.set_index('id', inplace=True)

The intersection will tell us the stations that are present in both dataframes. The result will be the index when performing an inner join:

In [ ]:
weather.index.intersection(station_info.index)

The set difference will tell us what we lose from each side. When performing an inner join, we lose nothing from the `weather` dataframe:

In [ ]:
weather.index.difference(station_info.index)

We lose 169 stations from the `station_info` dataframe, however:

In [ ]:
station_info.index.difference(weather.index)

The symmetric difference tells us what we lose from both sides. It is the combination of the set differences in each direction:

In [ ]:
ny_in_name = station_info[station_info.index.str.contains('US1NY')]

ny_in_name.index.difference(weather.index).shape[0]\
+ weather.index.difference(ny_in_name.index).shape[0]\
== weather.index.symmetric_difference(ny_in_name.index).shape[0]

The union will show us everything that will be present after a full outer join. Note that we pass in the unique values of the index to make sure we can see the number of stations we will be left with:

In [ ]:
weather.index.unique().union(station_info.index)

Note that the symmetric difference is actually the union of the set differences:

In [ ]:
ny_in_name = station_info[station_info.index.str.contains('US1NY')]

ny_in_name.index.difference(weather.index).union(weather.index.difference(ny_in_name.index)).equals(
    weather.index.symmetric_difference(ny_in_name.index)
)

<hr>
<div>
    <a href="../ch_03/5-handling_data_issues.ipynb">
        <button>&#8592; Lab 3</button>
    </a>
    <a href="./0-weather_data_collection.ipynb">
        <button>Weather Data Collection</button>
    </a>
    <a href="./2-dataframe_operations.ipynb">
        <button style="float: right;">Next Notebook &#8594;</button>
    </a>
</div>
<hr>